In [ ]:
import numpy as np
import sys
import matplotlib.pyplot as plt
from matplotlib import gridspec
import traceback

import iminuit
print("iminuit version:", iminuit.__version__)
from iminuit import Minuit
from iminuit.cost import LeastSquares
import inspect

import time
import os
import multiprocessing

import pickle

In [ ]:
def curve_fit(f, x, y, sigma=None, p0=None, p0_lim=None):

    if sigma is None:
        sigma = np.ones_like(y)

    init_params = {}
    if not p0 is None:
        for i, n in enumerate(inspect.getfullargspec(f)[0][1:]):
            init_params[n]=p0[i]
    if not p0_lim is None:
        for i, n in enumerate(inspect.getfullargspec(f)[0][1:]):
            init_params["limit_"+n]=p0_lim[i]

    least_squares = LeastSquares(x, y, sigma, f)

    m = Minuit(least_squares, **init_params)

    m.migrad()    
    try:
        cov = np.array(m.matrix())
    except:
        try:
            m.hesse()
            cov = np.array(m.matrix())
        except:
            m.hesse()
            cov = np.array(m.matrix())
           
    return m.np_values(), cov



In [ ]:
def avg_stepsize_corr_b1(positions, nominal_positions, outliers=0, od_ls=False, plot=False, r=1.0):

    nominal_step_size = nominal_positions[3] - nominal_positions[0]
    diffs =  np.diff( positions )

    if plot:
        plt.plot(range(len(diffs)), diffs, "o-b")

    if outliers == 0:
        avg_step_size = np.mean( diffs )
        od_std_b1     = np.std(diffs, ddof=1)
    else:
        excluded_indices = []

        diffs1  = np.array(diffs)
        for i in range(outliers):
            avg     = np.mean( diffs1 )
            i_extremum = np.argmax(np.abs(diffs1-avg))
            diffs1_ = diffs1[ np.arange(len(diffs1)) != i_extremum ]
            if np.abs(diffs1[i_extremum] - avg)<0.01:
                break
            elif np.std(diffs1_, ddof=1)/np.std(diffs1, ddof=1) > r:
                break
            excluded_indices.append( np.where(np.isclose(diffs, diffs1[i_extremum]))[0][0] )
            diffs1 = diffs1_
        
        avg = np.mean(diffs1)
        std = np.std(diffs1, ddof=1)
        od_std_b1 = std
        lim = 3*std
        diffs2 = diffs[ np.abs( diffs-avg )<=lim ]
        avg_step_size = np.mean(diffs2)

        if plot:
            plt.plot(excluded_indices, diffs[np.array(excluded_indices)], "xr", markersize=12)
            plt.plot([-1,5], [avg, avg], "-", color="red")
            plt.fill_between([-1,5], [avg-lim, avg-lim], [avg+lim, avg+lim], color="red", alpha=0.2)
            plt.plot(np.arange(len(diffs))[ np.abs( diffs-avg )>lim ]+0.3, diffs[ np.abs( diffs-avg )>lim ], "*m", markersize=12)
    
    if plot:
        plt.plot([-1,5], [avg_step_size, avg_step_size], "--g")
        plt.show()

            
    od_estimates = diffs - avg_step_size
    if od_ls:
        od_estimates *= nominal_step_size/avg_step_size
    new_stepsizes = od_estimates + nominal_step_size
    new_positions = np.concatenate(([0], np.cumsum(new_stepsizes)))
    # take the middle point as reference
    new_positions = new_positions - new_positions[2] + nominal_positions[7]

    return new_positions, od_std_b1


In [ ]:
def avg_stepsize_corr_sep(positions, nominal_positions, steps_to_average=10, outliers=0, od_ls=False, plot=False, r=1.0):

    nominal_step_size = nominal_positions[1] - nominal_positions[0]

    diffs = np.diff(positions)
    diffs = diffs[ (np.arange(len(diffs)) % 3) != 2]

    if plot:
        plt.plot(range(len(diffs)), diffs, "o-b")

    if steps_to_average == 10:
        if outliers == 0:
            avg_step_size = np.mean( diffs )
            od_std_sep    = np.std(diffs, ddof=1)
        else:
            # remove 4 outliers
            excluded_indices = []
            diffs1  =np.array(diffs)
            for i in range(outliers):
                avg  = np.mean( diffs1 )
                i_extremum = np.argmax(np.abs(diffs1-avg))
                diffs1_ = diffs1[ np.arange(len(diffs1)) != i_extremum ]
                if np.abs(diffs1[i_extremum] - avg)<0.01:
                    break
                elif np.std(diffs1_, ddof=1)/np.std(diffs1, ddof=1) > r:
                    break
                excluded_indices.append( np.where(np.isclose(diffs, diffs1[i_extremum]))[0][0] )
                diffs1 = diffs1_
            avg = np.mean(diffs1)
            std = np.std(diffs1, ddof=1)
            od_std_sep = std
            lim = 3*std
            diffs2 = diffs[ np.abs( diffs-avg )<=lim ]
            avg_step_size = np.mean(diffs2)


            if plot:

                plt.plot(excluded_indices, diffs[np.array(excluded_indices)], "xr", markersize=12)
                plt.plot([-1,11], [avg, avg], "-", color="red")
                plt.fill_between([-1,11], [avg-lim, avg-lim], [avg+lim, avg+lim], color="red", alpha=0.2)
                plt.plot(np.arange(len(diffs))[ np.abs( diffs-avg )>lim ]+0.3, diffs[ np.abs( diffs-avg )>lim ], "*m", markersize=12)
                
        if plot:
            plt.plot([-1,11], [avg_step_size, avg_step_size], "--g")
            plt.show()

        od_estimates = diffs - avg_step_size
        if od_ls:
            od_estimates *= nominal_step_size/avg_step_size
        new_stepsizes = od_estimates + nominal_step_size

    elif steps_to_average == 2:

        avg_step_sizes = np.repeat([ np.mean(diffs[2*i:2*i+2]) for i in range(5) ], 2)
        od_estimates = diffs - avg_step_sizes
        od_std_sep = np.std(od_estimates, ddof=1)
        if od_ls:
            od_estimates *= nominal_step_size/avg_step_sizes
        new_stepsizes = od_estimates + nominal_step_size

        if plot:
            for i in range(5):
                plt.plot(np.arange(len(avg_step_sizes))[2*i:2*i+2], avg_step_sizes[2*i:2*i+2], "--g")
            plt.show()

    # reference is always the middle point
    result = np.repeat(nominal_positions[1::3], 3)*1.0
    for i in range(len(result)):
        if i%3==0:
            result[i] -= new_stepsizes[i//3*2]
        elif i%3==2:
            result[i] += new_stepsizes[(i-2)//3*2+1]
    return result, od_std_sep



In [ ]:
def ls_stepsize_corr_b1(positions_b1, ls_b1_bpm_over_nominal):
    return positions_b1 * ls_b1_bpm_over_nominal, 0


def ls_stepsize_corr_sep(positions_b1, positions_b2, ls_b1_bpm_over_nominal, ls_b2_bpm_over_nominal):
    return positions_b1 * ls_b1_bpm_over_nominal - positions_b2 * ls_b2_bpm_over_nominal, 0


In [ ]:
def linfit_stepsize_corr_b1(positions_b1, nominal_positions_b1, positions_b1_5p, n_fit=5, od_ls=False):

    nominal_pos_5p = nominal_positions_b1[1::3]
    if n_fit == 5:
        fit_x   = nominal_pos_5p        
        fit_y   = positions_b1_5p        
    elif n_fit == 15:
        fit_x   = nominal_positions_b1        
        fit_y   = positions_b1
    else:
        raise ValueError("Unknown n_fit param "+str(n_fit))

    def lin(x,a,b):
        return a*x+b

    a0 = (fit_y[0] - fit_y[-1])/(fit_x[0] - fit_x[-1])
    b0 = fit_y[0] - fit_x[0]*a0
    popt_lin, pcov_lin = curve_fit(lin, fit_x,  fit_y, p0=[a0, b0])
    a = popt_lin[0]
    b = popt_lin[1]
    
    if not od_ls: # actually this would be better done the other way around
        new_pos_b1 = nominal_pos_5p + ( positions_b1_5p - (a*nominal_pos_5p+b) )
    else:
        new_pos_b1 = (positions_b1_5p-b)/a

    od_std_b1 = np.std(fit_y-lin(fit_x,a,b), ddof=1) * 1.41 # so that it is the same as the stepsize uncertainty in inf stat

    return new_pos_b1, od_std_b1


def linfit_stepsize_corr_sep(separations, nominal_separations, od_ls=False):
   
    def lin(x,a,b):
        return a*x+b

    a0 = (nominal_separations[0] - nominal_separations[-1])/(nominal_separations[0] - nominal_separations[-1])
    b0 = nominal_separations[0] - nominal_separations[0]*a0
    popt_lin, pcov_lin = curve_fit(lin, nominal_separations,  separations, p0=[a0, b0])
    a = popt_lin[0]
    b = popt_lin[1]

    if not od_ls: # actually this would be better done the other way around
        new_separations = nominal_separations + ( separations - (a*nominal_separations+b) )
    else:
        new_separations = (separations-b)/a

    od_std_sep = np.std(separations-lin(nominal_separations,a,b), ddof=1) * 1.41 # so that it is the same as the stepsize uncertainty in inf stat

    return new_separations, od_std_sep

In [ ]:
def get_headon_position_CMS(rates, rate_unc, separations, beamspot_positions, b1_position=None, bsp_unc=1 ):

    def sg(x,a,m,s):
        return a*np.exp(-0.5*((x-m)/s)**2)
    
    a0 = np.max(rates)
    m0 = np.average(separations, weights=rates)
    s0 = np.abs(separations[1] - separations[0])
    popt_sg, pcov_sg = curve_fit(sg, separations, rates, sigma=rate_unc, 
                                    p0=[a0, m0, s0 ],
                                    p0_lim = [
                                        [a0/2, a0*2],
                                        [m0-s0*3, m0+s0*3],
                                        [s0/2, s0*2],
                                    ] )

    mu               = popt_sg[1]

    def lin(x,a,b):
        return a*x+b

    popt_lin, pcov_lin = curve_fit(lin, separations, beamspot_positions, sigma=np.ones(3)*bsp_unc, p0=[1, 0])
    a = popt_lin[0]
    b = popt_lin[1]

    # compute result uncertainty
    if np.allclose(pcov_lin, 0):
        std = a*pcov_sg[1,1]**0.5
    else:
        mu_randomized = mu + pcov_sg[1,1]**0.5*np.random.normal(size=100)
        try:
            pcov_lin_sqrt = np.linalg.cholesky(pcov_lin)
        except Exception as e:
            print(e)
            print(pcov_lin)
            raise RuntimeError("Cannot perform Cholesky 1294")

        ab_randomized = popt_lin[:,np.newaxis] + np.einsum("ij,jk", pcov_lin_sqrt, np.random.normal(size=(2,100)))
        std = np.std( ab_randomized[0,:]*mu_randomized + ab_randomized[1,:], ddof=1 )

    if std == 0:
        std = 1e-6

    if not b1_position is None:
        ######
        # compute B1 position at head-on
        popt_lin, pcov_lin = curve_fit(lin, separations, b1_position, sigma=np.ones(3), p0=[1, 0])
        c = popt_lin[0]
        d = popt_lin[1]

        return a*mu+b, std, c*mu+d
    else:
        return a*mu+b, std, None


In [ ]:
def get_headon_position_ATLAS(rates, rate_unc, separations, beamspot_positions, b1_position=None, bsp_unc=1 ):

    beamspot_position_unc = np.ones(3)*bsp_unc

    def sg(x,a,m,s):
        return a*np.exp(-0.5*((x-m)/s)**2)

    def dsg_dx(x,a,m,s):
        return -(x-m)/s**2*sg(x,a,m,s)

    a0 = np.max(rates)
    m0 = np.average(beamspot_positions, weights=rates)
    s0 = np.abs(beamspot_positions[1] - beamspot_positions[0])
    popt_sg, pcov_sg = curve_fit(sg, beamspot_positions, rates, sigma=rate_unc, 
                                    p0=[a0, m0, s0 ],
                                    p0_lim = [
                                        [a0/2, a0*2],
                                        [m0-s0*3, m0+s0*3],
                                        [s0/2, s0*2],
                                    ] )
    a1 = popt_sg[0]
    m1 = popt_sg[1]
    s1 = popt_sg[2]
    combined_unc = (rate_unc**2 + (dsg_dx(beamspot_positions, a1,m1,s1)*beamspot_position_unc)**2)**0.5 
    popt_sg, pcov_sg = curve_fit(sg, beamspot_positions, rates, sigma=combined_unc, 
                                    p0=[a1, m1, s1 ],
                                    p0_lim = [
                                        [a1/2, a1*2],
                                        [m1-s1*3, m1+s1*3],
                                        [s1/2, s1*2],
                                    ] )                           

    mu              = popt_sg[1]
    mu_unc          = pcov_sg[1,1]**0.5

    if not b1_position is None:
        ######
        # compute B1 position at head-on

        # for nominal this fit would not work for example-->special case
        if np.abs(b1_position[-1]-b1_position[0]) < 0.1*np.abs(beamspot_positions[-1] - beamspot_positions[0]):
            return  mu, mu_unc, np.mean(b1_position)

        a0 = np.max(rates)
        m0 = np.average(b1_position, weights=rates)
        s0 = np.abs(b1_position[1] - b1_position[0])
        popt_sg, pcov_sg = curve_fit(sg, b1_position, rates, sigma=rate_unc, 
                                        p0=[a0, m0, s0 ],
                                        p0_lim = [
                                            [a0/2, a0*2],
                                            [m0-s0*3, m0+s0*3],
                                            [s0/2, s0*2],
                                        ] )
        mu_b1 = popt_sg[1]

        return  mu, mu_unc, mu_b1
    else:
        return mu, mu_unc, None


In [ ]:
def lsc(params):

    ## nominal beampositions
    nominal_B1_positions = np.repeat((np.arange(5)*params["nominal_step_size_B1"]),3)
    scan_pattern = np.tile([-params["nominal_step_size_B2"], 0, params["nominal_step_size_B2"]], 5)
    nominal_B2_positions = nominal_B1_positions + scan_pattern

    ## OD and BPM
    if params["cumulative_OD"]:
        true_B1_positions = nominal_B1_positions * params["Nominal_LS_B1"] + np.cumsum(params["OD_B1"])
        true_B2_positions = nominal_B2_positions * params["Nominal_LS_B2"] + np.cumsum(params["OD_B2"])
    else:
        true_B1_positions = nominal_B1_positions * params["Nominal_LS_B1"] + params["OD_B1"]
        true_B2_positions = nominal_B2_positions * params["Nominal_LS_B2"] + params["OD_B2"]

    doros_B1_positions = true_B1_positions / params["Doros_LS_B1"] + np.random.normal(size=true_B1_positions.shape) * params["Doros_noise_sigma"] 
    doros_B2_positions = true_B2_positions / params["Doros_LS_B2"] + np.random.normal(size=true_B2_positions.shape) * params["Doros_noise_sigma"]

    # arc_B1_positions = true_B1_positions / params["Arc_LS_B1"] + np.random.normal(size=true_B1_positions.shape) * params["Arc_noise_sigma"]
    # arc_B2_positions = true_B2_positions / params["Arc_LS_B2"] + np.random.normal(size=true_B2_positions.shape) * params["Arc_noise_sigma"]

    # avg_B1_positions = (doros_B1_positions + arc_B1_positions)/2
    # avg_B2_positions = (doros_B2_positions + arc_B2_positions)/2

    ## try a data drive OD estimation
    od_std_b1_full_naiive = np.std(doros_B1_positions - nominal_B1_positions, ddof=1)
    od_std_b2_full_naiive = np.std(doros_B2_positions - nominal_B2_positions, ddof=1)
    od_std_b1_full_naiive2 = np.std(np.diff(doros_B1_positions - nominal_B1_positions), ddof=1)
    od_std_b2_full_naiive2 = np.std(np.diff(doros_B2_positions - nominal_B2_positions), ddof=1)

    def lin(x,a,b):
        return a*x+b

    popt_lin, pcov_lin = curve_fit(lin, nominal_B1_positions,  doros_B1_positions, p0=[1, 0])
    od_std_b1_full_fit = np.std(doros_B1_positions - lin(nominal_B1_positions, *popt_lin), ddof=1)

    popt_lin, pcov_lin = curve_fit(lin, nominal_B2_positions,  doros_B2_positions, p0=[1, 0])
    od_std_b2_full_fit = np.std(doros_B2_positions - lin(nominal_B2_positions, *popt_lin), ddof=1)


    ## beamspot location 
    true_beamspot_positions = ( 
            true_B1_positions/params["Sigma_B1"]**2 + 
            true_B2_positions/params["Sigma_B2"]**2
        )/(params["Sigma_B1"]**-2+params["Sigma_B2"]**-2)

    if params["randomize_beamspot_position"]:
        randomized_beamspot_positions = true_beamspot_positions + np.random.normal(size=true_beamspot_positions.shape) * params["beamspot_position_uncertainty"]
    else:
        randomized_beamspot_positions = true_beamspot_positions
        
    ## rates
    true_rates       = np.exp(-0.5 * (true_B1_positions-true_B2_positions)**2 / (params["Sigma_B1"]**2+params["Sigma_B2"]**2) )
    true_rates_unc   = true_rates**0.5 * np.max(true_rates)**0.5 * params["rate_runcertainty_at_head_on"]

    if params["randomize_rates"]:
        randomized_rates = true_rates + np.random.normal(size=true_rates_unc.shape) * true_rates_unc
    else:
        randomized_rates = true_rates


    ##################### Analysis

    if params["pos_strategy"].startswith("doros, rel. ls based unc"):
        if "0.1%" in params["pos_strategy"]:
            ls_sigma = 0.001
        elif "0.25%" in params["pos_strategy"]:
            ls_sigma = 0.0025
        elif "0.5%" in params["pos_strategy"]:
            ls_sigma = 0.005
        elif "1%" in params["pos_strategy"]:
            ls_sigma = 0.01
        else:
            raise ValueError("Unexpected ls unc.")
        ls_b1_bpm_over_nominal = params["Doros_LS_B1"] / params["Nominal_LS_B1"] + np.random.normal()*ls_sigma
        ls_b2_bpm_over_nominal = params["Doros_LS_B2"] / params["Nominal_LS_B2"] + np.random.normal()*ls_sigma

    ## OD correction of separations
    if params["sep_strategy"]=="nominal":
        analysis_separations   = nominal_B1_positions - nominal_B2_positions
        diffs = np.diff(doros_B1_positions - doros_B2_positions)
        diffs = diffs[ (np.arange(len(diffs)) % 3) != 2]
        od_std_sep = np.std(diffs, ddof=1)
    ###
    elif params["sep_strategy"]=="doros":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        diffs = np.diff(analysis_separations)
        diffs = diffs[ (np.arange(len(diffs)) % 3) != 2]
        od_std_sep = np.std(diffs, ddof=1)
    ###
    elif params["sep_strategy"]=="doros, sep stepsize10":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=0)
    elif params["sep_strategy"]=="doros, sep stepsize2":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=2, outliers=0)
    elif params["sep_strategy"]=="doros, sep stepsize10 outlier2":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=2)
    elif params["sep_strategy"]=="doros, sep stepsize10 outlier2_0.50":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=2, r=0.5)
    elif params["sep_strategy"]=="doros, sep stepsize10 outlier4":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=4)
    elif params["sep_strategy"]=="doros, sep stepsize10 outlier4_0.50":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=4, r=0.5)
    ###
    elif params["sep_strategy"]=="doros, sep stepsize10, od_ls":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=0, od_ls=True)
    elif params["sep_strategy"]=="doros, sep stepsize2, od_ls":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=2, outliers=0, od_ls=True)
    elif params["sep_strategy"]=="doros, sep stepsize10 outlier2, od_ls":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=2, od_ls=True)
    elif params["sep_strategy"]=="doros, sep stepsize10 outlier2_0.50, od_ls":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=2, od_ls=True, r=0.5)
    elif params["sep_strategy"]=="doros, sep stepsize10 outlier4, od_ls":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=4, od_ls=True)
    elif params["sep_strategy"]=="doros, sep stepsize10 outlier4_0.50, od_ls":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=4, od_ls=True, r=0.5)
    ###
    elif params["sep_strategy"].startswith("doros, rel. ls based unc"):  
        analysis_separations, od_std_sep   = ls_stepsize_corr_sep(doros_B1_positions, doros_B2_positions, ls_b1_bpm_over_nominal, ls_b2_bpm_over_nominal)
    ###
    elif params["sep_strategy"]=="doros, sep linfit":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = linfit_stepsize_corr_sep(analysis_separations, nominal_B1_positions-nominal_B2_positions)
    elif params["sep_strategy"]=="doros, sep linfit, od_ls":
        analysis_separations   = doros_B1_positions - doros_B2_positions
        analysis_separations, od_std_sep   = linfit_stepsize_corr_sep(analysis_separations, nominal_B1_positions-nominal_B2_positions, od_ls=True )
    else:
        raise Exception("Sep strategy unknown "+params["sep_strategy"])   

    

    ## calculate head-on beamspot position

    doros_B1_5positions_fit         = []
    computed_head_on_bs_position    = []
    computed_head_on_bs_position_unc= []
    for i in range(5):

        doros_B1_3positions = None
        if params["miniscan_b1_strategy"]=="fit":
            doros_B1_3positions = doros_B1_positions[3*i:3*i+3]

        if params["headon_determination_method"] == "CMS":
            v, u, w = get_headon_position_CMS(
                randomized_rates[3*i:3*i+3],
                true_rates_unc[3*i:3*i+3],
                analysis_separations[3*i:3*i+3],
                randomized_beamspot_positions[3*i:3*i+3],
                doros_B1_3positions,
                params["beamspot_position_uncertainty"])

            doros_B1_5positions_fit.append(w)

        elif params["headon_determination_method"] == "ATLAS":
            v, u, w = get_headon_position_ATLAS(
                randomized_rates[3*i:3*i+3],
                true_rates_unc[3*i:3*i+3],
                analysis_separations[3*i:3*i+3],
                randomized_beamspot_positions[3*i:3*i+3],
                None,
                params["beamspot_position_uncertainty"])

            doros_B1_5positions_fit.append(w)

        else:
            raise ValueError("Unknown headon_determination_method " + params["headon_determination_method"])

        if not (np.isfinite(v) and np.isfinite(u)):
            print(params["headon_determination_method"])
            print(i)
            print(u, v)
            print([
                randomized_rates[3*i:3*i+3],
                true_rates_unc[3*i:3*i+3],
                analysis_separations[3*i:3*i+3],
                randomized_beamspot_positions[3*i:3*i+3],
                doros_B1_positions[3*i:3*i+3],
                params["beamspot_position_uncertainty"]
            ])
            print(true_B1_positions-true_B2_positions)
            print(analysis_separations)
            raise RuntimeError("u or v not finite 897")

        computed_head_on_bs_position.append(v) 
        computed_head_on_bs_position_unc.append(u)

    doros_B1_5positions_fit = np.array(doros_B1_5positions_fit)


    # B1 handling and corrections
    if params["b1_strategy"] == "nominal":
        analysis_B1_5positions = nominal_B1_positions[1::3]
        od_std_b1 = np.std(np.diff([ np.mean(doros_B1_positions[3*i:3*i+3]) for i in range(5)]), ddof=1)
    else:
        if params["miniscan_b1_strategy"]=="avg":
            doros_B1_5positions   = np.array([ np.mean(doros_B1_positions[3*i:3*i+3]) for i in range(5)])
        elif params["miniscan_b1_strategy"]=="middle":
            doros_B1_5positions   = doros_B1_positions[1::3]
        elif params["miniscan_b1_strategy"]=="fit":
            doros_B1_5positions   = doros_B1_5positions_fit
        else:
            raise Exception("Miniscan b1 strategy unknown "+params["miniscan_b1_strategy"]) 
    
        if params["b1_strategy"] == "doros":
            analysis_B1_5positions = doros_B1_5positions
            od_std_b1 = np.std(np.diff(doros_B1_5positions), ddof=1)
        ###
        elif params["b1_strategy"]=="doros, b1 stepsize":
            analysis_B1_5positions, od_std_b1 = avg_stepsize_corr_b1(doros_B1_5positions, nominal_B1_positions, outliers=0)
        elif params["b1_strategy"]=="doros, b1 stepsize outlier1":
            analysis_B1_5positions, od_std_b1 = avg_stepsize_corr_b1(doros_B1_5positions, nominal_B1_positions, outliers=1)
        elif params["b1_strategy"]=="doros, b1 stepsize outlier1_0.35":
            analysis_B1_5positions, od_std_b1 = avg_stepsize_corr_b1(doros_B1_5positions, nominal_B1_positions, outliers=1, r=0.35)
        elif params["b1_strategy"]=="doros, b1 stepsize outlier2":
            analysis_B1_5positions, od_std_b1 = avg_stepsize_corr_b1(doros_B1_5positions, nominal_B1_positions, outliers=2)
        elif params["b1_strategy"]=="doros, b1 stepsize outlier2_0.35":
            analysis_B1_5positions, od_std_b1 = avg_stepsize_corr_b1(doros_B1_5positions, nominal_B1_positions, outliers=2, r=0.35)
        ###
        elif params["b1_strategy"]=="doros, b1 stepsize, od_ls":
            analysis_B1_5positions, od_std_b1 = avg_stepsize_corr_b1(doros_B1_5positions, nominal_B1_positions, outliers=0, od_ls = True)
        elif params["b1_strategy"]=="doros, b1 stepsize outlier1, od_ls":
            analysis_B1_5positions, od_std_b1 = avg_stepsize_corr_b1(doros_B1_5positions, nominal_B1_positions, outliers=1, od_ls = True)
        elif params["b1_strategy"]=="doros, b1 stepsize outlier1_0.35, od_ls":
            analysis_B1_5positions, od_std_b1 = avg_stepsize_corr_b1(doros_B1_5positions, nominal_B1_positions, outliers=1, od_ls = True, r=0.35)
        elif params["b1_strategy"]=="doros, b1 stepsize outlier2, od_ls":
            analysis_B1_5positions, od_std_b1 = avg_stepsize_corr_b1(doros_B1_5positions, nominal_B1_positions, outliers=2, od_ls = True)
        elif params["b1_strategy"]=="doros, b1 stepsize outlier2_0.35, od_ls":
            analysis_B1_5positions, od_std_b1 = avg_stepsize_corr_b1(doros_B1_5positions, nominal_B1_positions, outliers=2, od_ls = True, r=0.35)
        ###
        elif params["b1_strategy"].startswith("doros, rel. ls based unc"): 
            analysis_B1_5positions, od_std_b1 = ls_stepsize_corr_b1(doros_B1_5positions, ls_b1_bpm_over_nominal)
        ###
        elif params["b1_strategy"]=="doros, b1 linfit5":
            analysis_B1_5positions, od_std_b1 = linfit_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, doros_B1_5positions, n_fit=5 )
        elif params["b1_strategy"]=="doros, b1 linfit5, od_ls":
            analysis_B1_5positions, od_std_b1 = linfit_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, doros_B1_5positions, n_fit=5, od_ls = True )
        elif params["b1_strategy"]=="doros, b1 linfit15":
            analysis_B1_5positions, od_std_b1 = linfit_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, doros_B1_5positions, n_fit=15)
        elif params["b1_strategy"]=="doros, b1 linfit15, od_ls":
            analysis_B1_5positions, od_std_b1 = linfit_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, doros_B1_5positions, n_fit=15, od_ls = True )
        else:
            raise Exception("Bb1 strategy unknown "+params["b1_strategy"]) 

    ## fit linear function
    def lin(x,a,b):
        return a*x+b

    a0 = (computed_head_on_bs_position[-1]-computed_head_on_bs_position[0])/(analysis_B1_5positions[-1]-analysis_B1_5positions[0])
    b0 = 0
    popt_lin, pcov_lin = curve_fit(lin, analysis_B1_5positions, computed_head_on_bs_position, sigma=computed_head_on_bs_position_unc, p0=[a0, b0])
    a = popt_lin[0]
    b = popt_lin[1]

    if not (np.isfinite(a) and np.isfinite(b)):
        print(a, b)
        print([
            analysis_B1_5positions,
            computed_head_on_bs_position,
        ])
        raise Exception("reuslt is not finite")
        
    return [
                a, pcov_lin[0,0]**0.5, 
                od_std_b1, od_std_sep, 
                od_std_b1_full_naiive, od_std_b2_full_naiive, 
                od_std_b1_full_naiive2, od_std_b2_full_naiive2, 
                od_std_b1_full_fit, od_std_b2_full_fit
    ]

In [ ]:
def worker(argv):

    params, N, plot = argv

    OD_distribution = params["OD_distribution"]

    settings_txt = "\n".join([
        "Settings:",
        "position handling strategy:\n  " + "\n  ".join([
            params["sep_strategy"],
            params["miniscan_b1_strategy"],
            params["b1_strategy"],
        ]),
        "head-on determination method: {}".format(params["headon_determination_method"]),
        "Nominal LS B1: {}".format(params["Nominal_LS_B1"]),
        "Nominal LS B2: {}".format(params["Nominal_LS_B2"]),
        "Doros LS B1: {}".format(params["Doros_LS_B1"]),
        "Doros LS B2: {}".format(params["Doros_LS_B2"]),
        # "Arc_LS_B1: {}".format(params["Arc_LS_B1"]),
        # "Arc_LS_B2: {}".format(params["Arc_LS_B2"]),
        "Sigma B1: {} um".format(params["Sigma_B1"]),
        "Sigma B2: {} um".format(params["Sigma_B2"]),
        "nominal step size B1: {} um".format(params["nominal_step_size_B1"]),
        "nominal step size B2: {} um".format(params["nominal_step_size_B2"]),
        "OD distribution: {}".format(OD_distribution),
        "Markov Chain OD: {}".format(params["cumulative_OD"]),
        "Doros noise_sigma: {} um".format(params["Doros_noise_sigma"]),
        "rate runcertainty at head on: {}%".format(params["rate_runcertainty_at_head_on"]*100),
        "randomize rates: {}".format(params["randomize_rates"]),
        "beamspot position uncertainty: {}".format(params["beamspot_position_uncertainty"]),
        "randomize beamspot position: {}".format(params["randomize_beamspot_position"]),
    ])

   
    ls_results = []
    ls_result_errs = []
    ls_result_od_b1_errs = []
    ls_result_od_sep_errs = []
    ls_result_od_std_b1_full_naiive = []
    ls_result_od_std_b2_full_naiive = []
    ls_result_od_std_b1_full_naiive2 = []
    ls_result_od_std_b2_full_naiive2 = []
    ls_result_od_std_b1_full_fit = []
    ls_result_od_std_b2_full_fit = []
    for i in range(N):

        np.random.seed(i)

        if OD_distribution[0] == "Normal":
            sigma_OD = OD_distribution[1]
            params.update(
                {
                    "OD_B1":np.random.normal(size=15)*sigma_OD,
                    "OD_B2":np.random.normal(size=15)*sigma_OD,
                })
        elif OD_distribution[0] == "NormalSum":
            p_OD = OD_distribution[1]
            sigma1_OD = OD_distribution[2]
            sigma2_OD = OD_distribution[3]
            mask  = np.random.random(size=15)<p_OD
            od_b1 = np.random.normal(size=15)*sigma1_OD * mask  + np.random.normal(size=15)*sigma2_OD * (1-mask)
            mask  = np.random.random(size=15)<p_OD
            od_b2 = np.random.normal(size=15)*sigma1_OD * mask  + np.random.normal(size=15)*sigma2_OD * (1-mask)
            params.update(
                {
                    "OD_B1":od_b1,
                    "OD_B2":od_b2,
                })
        elif OD_distribution[0] == "AsymNormal":
            sigma1_OD = OD_distribution[1]
            sigma2_OD = OD_distribution[2]
            params.update(
                {
                    "OD_B1":np.random.normal(size=15)*sigma1_OD,
                    "OD_B2":np.random.normal(size=15)*sigma2_OD,
                })
        elif OD_distribution[0] == "NormalB1kick":
            sigma_OD = OD_distribution[1]
            od_b1 = np.random.normal(size=15)*sigma_OD
            od_b2 = np.random.normal(size=15)*sigma_OD
            od_b1[int(np.random.random()*15)] = OD_distribution[2]
            params.update(
                {
                    "OD_B1":od_b1,
                    "OD_B2":od_b2,
                })
        elif OD_distribution[0] == "NormalB1skick":
            sigma_OD = OD_distribution[1]
            od_b1 = np.random.normal(size=15)*sigma_OD
            od_b2 = np.random.normal(size=15)*sigma_OD
            od_b1[int(np.random.random()*15)] = OD_distribution[2] * (-1)**int(np.random.random()*2)
            params.update(
                {
                    "OD_B1":od_b1,
                    "OD_B2":od_b2,
                })
        elif OD_distribution[0] == "NormalB2kick":
            sigma_OD = OD_distribution[1]
            od_b1 = np.random.normal(size=15)*sigma_OD
            od_b2 = np.random.normal(size=15)*sigma_OD
            od_b2[int(np.random.random()*15)] = OD_distribution[2]
            # print(od_b1)
            # print(od_b2)
            params.update(
                {
                    "OD_B1":od_b1,
                    "OD_B2":od_b2,
                })
        elif OD_distribution[0] == "NormalB2skick":
            sigma_OD = OD_distribution[1]
            od_b1 = np.random.normal(size=15)*sigma_OD
            od_b2 = np.random.normal(size=15)*sigma_OD
            od_b2[int(np.random.random()*15)] = OD_distribution[2] * (-1)**int(np.random.random()*2)
            # print(od_b1)
            # print(od_b2)
            params.update(
                {
                    "OD_B1":od_b1,
                    "OD_B2":od_b2,
                })
        elif OD_distribution[0] == "CauchyLim":
            gamma_OD = OD_distribution[1]
            lim = OD_distribution[2]

            od_b1 = np.random.standard_cauchy(size=15)*gamma_OD
            od_b1 = np.clip( od_b1, a_min=-lim, a_max=lim)
            od_b2 = np.random.standard_cauchy(size=15)*gamma_OD
            od_b2 = np.clip( od_b2, a_min=-lim, a_max=lim)

            params.update(
                {
                    "OD_B1":od_b1,
                    "OD_B2":od_b2,
                })
        else:
            raise ValueError("Unknown OD model.")

        try:
            ls, lse, \
            od_std_b1, od_std_sep, \
            od_std_b1_full_naiive, od_std_b2_full_naiive, \
            od_std_b1_full_naiive2, od_std_b2_full_naiive2, \
            od_std_b1_full_fit, od_std_b2_full_fit = lsc(params)
            ls_results.append(ls)
            ls_result_errs.append(lse)
            ls_result_od_b1_errs.append(od_std_b1)
            ls_result_od_sep_errs.append(od_std_sep)
            ls_result_od_std_b1_full_naiive.append(od_std_b1_full_naiive)
            ls_result_od_std_b2_full_naiive.append(od_std_b2_full_naiive)
            ls_result_od_std_b1_full_naiive2.append(od_std_b1_full_naiive2)
            ls_result_od_std_b2_full_naiive2.append(od_std_b2_full_naiive2)
            ls_result_od_std_b1_full_fit.append(od_std_b1_full_fit)
            ls_result_od_std_b2_full_fit.append(od_std_b2_full_fit)
        except Exception as e:
            print("Error escaped worker function.")
            print("Error:")
            print(e)
            print("Args:")
            print(argv)
            print("Traceback:")
            print(traceback.print_exc())
            

            with open("err.txt", "w") as f:
                f.write("{}\n{}\n{}".format(argv, traceback.print_exc(), e))

            raise RuntimeError("Error in worker function.")


    if plot:
        fig = plt.figure(figsize=(10,10))
        fig.patch.set_facecolor('white')
        gs = gridspec.GridSpec(2,2)
        # gs.update(hspace=0.1)

        ax0  = fig.add_subplot(gs[0,0])
        ax0.set_xlabel("LS", fontsize=16)
        ax1 = fig.add_subplot(gs[1,0])
        ax1.set_xlabel("LS unc $\\times$ 100", fontsize=16)
        for ax in [ax0, ax1]:
            ax.grid(True)
            ax.tick_params(axis='y', which='major', direction="in", labelsize=16, pad = 8)
            ax.tick_params(axis='x', which='major', direction="in", labelsize=16, pad = 12)
            for axis in ['top','bottom','left','right']:
                ax.spines[axis].set_linewidth(0.5)

        ax2 = fig.add_subplot(gs[:,1])   
        for axis in ['top','bottom','left','right']:
            ax2.spines[axis].set_linewidth(0.0)
        ax2.tick_params(
                axis='both',          
                which='both',     
                bottom=False,      
                top=False, 
                left=False,      
                right=False,         
                labelbottom=False,
                labelleft=False)

        

        ax0.hist(ls_results, 50, density=False, facecolor='b', alpha=0.75)
        ax1.hist(ls_result_errs*100, 50, density=False, facecolor='g', alpha=0.75)
        # ax0.hist((np.array(ls_results)-1)*100, 50, density=False, facecolor='b', alpha=0.75)
        # ax1.hist((np.array(ls_results)-1)*1000, 50, density=False, facecolor='g', alpha=0.75)


        ax0.set_ylim(0, ax0.get_ylim()[1]*1.15)
        ax0.text(0.02, 0.97, "Mean: {:0.4f}$\\pm${:0.4f}\nStd:{:0.4f}".format(np.mean(ls_results), np.std(ls_results, ddof=1)/N**0.5, np.std(ls_results, ddof=1)),
                        horizontalalignment='left',
                        verticalalignment='top',
                        transform=ax0.transAxes,
                        fontname='sans-serif',
                        fontweight='bold',
                        fontsize=14)

        ax1.set_ylim(0, ax1.get_ylim()[1]*1.15)
        ax1.text(0.02, 0.97, "Mean: {:0.4f}$\\pm${:0.4f}\nStd:{:0.4f}".format(np.mean(ls_result_errs), np.std(ls_result_errs, ddof=1)/N**0.5,  np.std(ls_result_errs, ddof=1)),
                        horizontalalignment='left',
                        verticalalignment='top',
                        transform=ax1.transAxes,
                        fontname='sans-serif',
                        fontweight='bold',
                        fontsize=14)

        ax2.text(0.02, 0.97, settings_txt,
                        horizontalalignment='left',
                        verticalalignment='top',
                        transform=ax2.transAxes,
                        fontname='sans-serif',
                        fontweight='normal',
                        fontsize=14)

        fig.savefig("OD_{}_cumulative_{}_strategy_{}_{}.png".format(
            OD_distribution, 
            cumulative_OD, 
            params["pos_strategy"],
            params["headon_determination_method"]
            ))
        plt.close(fig)

    return [
            params["cumulative_OD"], 
            params["OD_distribution"], 
            params["pos_strategy"], 
            params["headon_determination_method"], 
            #
            ls_results, ls_result_errs, 
            ls_result_od_b1_errs, ls_result_od_sep_errs, 
            ls_result_od_std_b1_full_naiive, ls_result_od_std_b2_full_naiive,
            ls_result_od_std_b1_full_naiive2, ls_result_od_std_b2_full_naiive2,
            ls_result_od_std_b1_full_fit, ls_result_od_std_b2_full_fit
            ]

In [ ]:
if False:

    params = {
        "Nominal_LS_B1":0.9975,
        "Nominal_LS_B2":0.995,
        "Doros_LS_B1":1.0025,
        "Doros_LS_B2":1.005,
        "Arc_LS_B1":1.005,
        "Arc_LS_B2":1.01,
        "Sigma_B1":90, #micron
        "Sigma_B2":90, #micron
        "nominal_step_size_B1": 130, #micron
        "nominal_step_size_B2": 130, #micron
        "Doros_noise_sigma":0.25, # micron
        "Arc_noise_sigma":0, # micron
        "rate_runcertainty_at_head_on":0.005, #relative
        "randomize_rates":True,
        "beamspot_position_uncertainty":0.25, #micron
        "randomize_beamspot_position":True,
        ##
        "cumulative_OD":True,
        "OD_distribution":("Normal",2),
        "headon_determination_method":"CMS",
        #
        "b1_strategy":"doros, b1 linfit",
        "miniscan_b1_strategy":"fit",
        "sep_strategy":"doros",
        "pos_strategy":"whatever",
        #
    }

    _ = worker([params, 10, False])

In [ ]:
N = 10000
make_plots = True
parallel_execution = True

params = {
    "Nominal_LS_B1":0.9975,
    "Nominal_LS_B2":0.995,
    "Doros_LS_B1":1.0025,
    "Doros_LS_B2":1.005,
    "Arc_LS_B1":1.005,
    "Arc_LS_B2":1.01,
    "Sigma_B1":90, #micron
    "Sigma_B2":90, #micron
    "nominal_step_size_B1": 130, #micron
    "nominal_step_size_B2": 130, #micron
    "Doros_noise_sigma":0.25, # micron, statistical fluctuation on top of true value
    "Arc_noise_sigma":0, # micron
    "rate_runcertainty_at_head_on":0.005, #relative
    "randomize_rates":True,
    "beamspot_position_uncertainty":0.25, #micron, looks like 0.15 um in Mahmoud's fits btw
    "randomize_beamspot_position":True,
}

pos_strategies = { # b1_strategy, miniscan_b1_strategy, sep_strategy
    "nominal, middle point":("nominal", "middle", "nominal"),
    #
    "doros, middle point":  ("doros", "middle", "doros"),
    "doros, avg point":     ("doros", "avg", "doros"),
    "doros, fit point":     ("doros", "fit", "doros"),
    #
    "doros, b1 stepsize, sep stepsize10, avg point":("doros, b1 stepsize", "avg", "doros, sep stepsize10"),
    "doros, b1 stepsize, sep stepsize2, avg point":("doros, b1 stepsize", "avg", "doros, sep stepsize2"),
    "doros, b1 stepsize, sep stepsize10 outlier4, avg point":("doros, b1 stepsize", "avg", "doros, sep stepsize10 outlier4"),
    "doros, b1 stepsize outlier1, sep stepsize10, avg point":("doros, b1 stepsize outlier1", "avg", "doros, sep stepsize10"),
    "doros, b1 stepsize outlier1, sep stepsize10 outlier4, avg point":("doros, b1 stepsize outlier1", "avg", "doros, sep stepsize10 outlier4"),
    "doros, b1 stepsize outlier1, sep stepsize10 outlier2, avg point":("doros, b1 stepsize outlier1", "avg", "doros, sep stepsize10 outlier2"),
    "doros, b1 stepsize outlier2, sep stepsize10, avg point":("doros, b1 stepsize outlier2", "avg", "doros, sep stepsize10"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier4, avg point":("doros, b1 stepsize outlier2", "avg", "doros, sep stepsize10 outlier4"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier2, avg point":("doros, b1 stepsize outlier2", "avg", "doros, sep stepsize10 outlier2"),
    #
    "doros, b1 stepsize outlier1_0.35, sep stepsize10, avg point":("doros, b1 stepsize outlier1_0.35", "avg", "doros, sep stepsize10"),
    "doros, b1 stepsize outlier1_0.35, sep stepsize10 outlier4, avg point":("doros, b1 stepsize outlier1_0.35", "avg", "doros, sep stepsize10 outlier4"),
    "doros, b1 stepsize outlier1_0.35, sep stepsize10 outlier2, avg point":("doros, b1 stepsize outlier1_0.35", "avg", "doros, sep stepsize10 outlier2"),
    "doros, b1 stepsize outlier2_0.35, sep stepsize10, avg point":("doros, b1 stepsize outlier2_0.35", "avg", "doros, sep stepsize10"),
    "doros, b1 stepsize outlier2_0.35, sep stepsize10 outlier4, avg point":("doros, b1 stepsize outlier2_0.35", "avg", "doros, sep stepsize10 outlier4"),
    "doros, b1 stepsize outlier2_0.35, sep stepsize10 outlier2, avg point":("doros, b1 stepsize outlier2_0.35", "avg", "doros, sep stepsize10 outlier2"),
    #
    "doros, b1 stepsize outlier1, sep stepsize10 outlier4_0.50, avg point":("doros, b1 stepsize outlier1", "avg", "doros, sep stepsize10 outlier4_0.50"),
    "doros, b1 stepsize outlier1, sep stepsize10 outlier2_0.50, avg point":("doros, b1 stepsize outlier1", "avg", "doros, sep stepsize10 outlier2_0.50"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier4_0.50, avg point":("doros, b1 stepsize outlier2", "avg", "doros, sep stepsize10 outlier4_0.50"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier2_0.50, avg point":("doros, b1 stepsize outlier2", "avg", "doros, sep stepsize10 outlier2_0.50"),
    #
    "doros, b1 stepsize outlier1_0.35, sep stepsize10 outlier4_0.50, avg point":("doros, b1 stepsize outlier1_0.35", "avg", "doros, sep stepsize10 outlier4_0.50"),
    "doros, b1 stepsize outlier1_0.35, sep stepsize10 outlier2_0.50, avg point":("doros, b1 stepsize outlier1_0.35", "avg", "doros, sep stepsize10 outlier2_0.50"),
    "doros, b1 stepsize outlier2_0.35, sep stepsize10 outlier4_0.50, avg point":("doros, b1 stepsize outlier2_0.35", "avg", "doros, sep stepsize10 outlier4_0.50"),
    "doros, b1 stepsize outlier2_0.35, sep stepsize10 outlier2_0.50, avg point":("doros, b1 stepsize outlier2_0.35", "avg", "doros, sep stepsize10 outlier2_0.50"),
    #
    "doros, b1 stepsize, sep stepsize10, middle point":("doros, b1 stepsize", "middle", "doros, sep stepsize10"),
    "doros, b1 stepsize, sep stepsize2, middle point":("doros, b1 stepsize", "middle", "doros, sep stepsize2"),
    "doros, b1 stepsize, sep stepsize10 outlier4, middle point":("doros, b1 stepsize", "middle", "doros, sep stepsize10 outlier4"),
    "doros, b1 stepsize outlier1, sep stepsize10, middle point":("doros, b1 stepsize outlier1", "middle", "doros, sep stepsize10"),
    "doros, b1 stepsize outlier1, sep stepsize10 outlier4, middle point":("doros, b1 stepsize outlier1", "middle", "doros, sep stepsize10 outlier4"),
    "doros, b1 stepsize outlier1, sep stepsize10 outlier2, middle point":("doros, b1 stepsize outlier1", "middle", "doros, sep stepsize10 outlier2"),
    "doros, b1 stepsize outlier2, sep stepsize10, middle point":("doros, b1 stepsize outlier2", "middle", "doros, sep stepsize10"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier4, middle point":("doros, b1 stepsize outlier2", "middle", "doros, sep stepsize10 outlier4"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier2, middle point":("doros, b1 stepsize outlier2", "middle", "doros, sep stepsize10 outlier2"),
    #
    "doros, b1 stepsize, sep stepsize10, fit point":("doros, b1 stepsize", "fit", "doros, sep stepsize10"),
    "doros, b1 stepsize, sep stepsize2, fit point":("doros, b1 stepsize", "fit", "doros, sep stepsize2"),
    "doros, b1 stepsize, sep stepsize10 outlier4, fit point":("doros, b1 stepsize", "fit", "doros, sep stepsize10 outlier4"),
    "doros, b1 stepsize outlier1, sep stepsize10, fit point":("doros, b1 stepsize outlier1", "fit", "doros, sep stepsize10"),
    "doros, b1 stepsize outlier1, sep stepsize10 outlier4, fit point":("doros, b1 stepsize outlier1", "fit", "doros, sep stepsize10 outlier4"),
    "doros, b1 stepsize outlier1, sep stepsize10 outlier2, fit point":("doros, b1 stepsize outlier1", "fit", "doros, sep stepsize10 outlier2"),
    "doros, b1 stepsize outlier2, sep stepsize10, fit point":("doros, b1 stepsize outlier2", "fit", "doros, sep stepsize10"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier4, fit point":("doros, b1 stepsize outlier2", "fit", "doros, sep stepsize10 outlier4"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier2, fit point":("doros, b1 stepsize outlier2", "fit", "doros, sep stepsize10 outlier2"),
    #
    "doros, b1 stepsize, sep stepsize10, avg point, od_ls":("doros, b1 stepsize", "avg", "doros, sep stepsize10, od_ls"),
    "doros, b1 stepsize, sep stepsize2, avg point, od_ls":("doros, b1 stepsize", "avg", "doros, sep stepsize2, od_ls"),
    "doros, b1 stepsize, sep stepsize10 outlier4, avg point, od_ls":("doros, b1 stepsize", "avg", "doros, sep stepsize10 outlier4, od_ls"),
    "doros, b1 stepsize outlier1, sep stepsize10, avg point, od_ls":("doros, b1 stepsize outlier1", "avg", "doros, sep stepsize10, od_ls"),
    "doros, b1 stepsize outlier1, sep stepsize10 outlier4, avg point, od_ls":("doros, b1 stepsize outlier1", "avg", "doros, sep stepsize10 outlier4, od_ls"),
    "doros, b1 stepsize outlier1, sep stepsize10 outlier2, avg point, od_ls":("doros, b1 stepsize outlier1", "avg", "doros, sep stepsize10 outlier2, od_ls"),
    "doros, b1 stepsize outlier2, sep stepsize10, avg point, od_ls":("doros, b1 stepsize outlier2", "avg", "doros, sep stepsize10, od_ls"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier4, avg point, od_ls":("doros, b1 stepsize outlier2", "avg", "doros, sep stepsize10 outlier4, od_ls"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier2, avg point, od_ls":("doros, b1 stepsize outlier2", "avg", "doros, sep stepsize10 outlier2, od_ls"),
    #
    "doros, b1 stepsize, sep stepsize10, middle point, od_ls":("doros, b1 stepsize", "middle", "doros, sep stepsize10, od_ls"),
    "doros, b1 stepsize, sep stepsize2, middle point, od_ls":("doros, b1 stepsize", "middle", "doros, sep stepsize2, od_ls"),
    "doros, b1 stepsize, sep stepsize10 outlier4, middle point, od_ls":("doros, b1 stepsize", "middle", "doros, sep stepsize10 outlier4, od_ls"),
    "doros, b1 stepsize outlier1, sep stepsize10, middle point, od_ls":("doros, b1 stepsize outlier1", "middle", "doros, sep stepsize10, od_ls"),
    "doros, b1 stepsize outlier1, sep stepsize10 outlier4, middle point, od_ls":("doros, b1 stepsize outlier1", "middle", "doros, sep stepsize10 outlier4, od_ls"),
    "doros, b1 stepsize outlier1, sep stepsize10 outlier2, middle point, od_ls":("doros, b1 stepsize outlier1", "middle", "doros, sep stepsize10 outlier2, od_ls"),
    "doros, b1 stepsize outlier2, sep stepsize10, middle point, od_ls":("doros, b1 stepsize outlier2", "middle", "doros, sep stepsize10, od_ls"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier4, middle point, od_ls":("doros, b1 stepsize outlier2", "middle", "doros, sep stepsize10 outlier4, od_ls"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier2, middle point, od_ls":("doros, b1 stepsize outlier2", "middle", "doros, sep stepsize10 outlier2, od_ls"),
    #
    "doros, b1 stepsize, sep stepsize10, fit point, od_ls":("doros, b1 stepsize", "fit", "doros, sep stepsize10, od_ls"),
    "doros, b1 stepsize, sep stepsize2, fit point, od_ls":("doros, b1 stepsize", "fit", "doros, sep stepsize2, od_ls"),
    "doros, b1 stepsize, sep stepsize10 outlier4, fit point, od_ls":("doros, b1 stepsize", "fit", "doros, sep stepsize10 outlier4, od_ls"),
    "doros, b1 stepsize outlier1, sep stepsize10, fit point, od_ls":("doros, b1 stepsize outlier1", "fit", "doros, sep stepsize10, od_ls"),
    "doros, b1 stepsize outlier1, sep stepsize10 outlier4, fit point, od_ls":("doros, b1 stepsize outlier1", "fit", "doros, sep stepsize10 outlier4, od_ls"),
    "doros, b1 stepsize outlier1, sep stepsize10 outlier2, fit point, od_ls":("doros, b1 stepsize outlier1", "fit", "doros, sep stepsize10 outlier2, od_ls"),
    "doros, b1 stepsize outlier2, sep stepsize10, fit point, od_ls":("doros, b1 stepsize outlier2", "fit", "doros, sep stepsize10, od_ls"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier4, fit point, od_ls":("doros, b1 stepsize outlier2", "fit", "doros, sep stepsize10 outlier4, od_ls"),
    "doros, b1 stepsize outlier2, sep stepsize10 outlier2, fit point, od_ls":("doros, b1 stepsize outlier2", "fit", "doros, sep stepsize10 outlier2, od_ls"),
    #
    "doros, b1 stepsize, avg point":("doros, b1 stepsize", "avg", "doros"),
    "doros, b1 stepsize outlier1, avg point":("doros, b1 stepsize outlier1", "avg", "doros"),
    "doros, b1 stepsize outlier1_0.35, avg point":("doros, b1 stepsize outlier1_0.35", "avg", "doros"),
    "doros, b1 stepsize, middle point":("doros, b1 stepsize", "middle", "doros"),
    "doros, b1 stepsize outlier1, middle point":("doros, b1 stepsize outlier1", "middle", "doros"),
    "doros, b1 stepsize outlier1_0.35, middle point":("doros, b1 stepsize outlier1_0.35", "middle", "doros"),
    "doros, b1 stepsize, fit point":("doros, b1 stepsize", "fit", "doros"),
    "doros, b1 stepsize outlier1, fit point":("doros, b1 stepsize outlier1", "fit", "doros"),
    "doros, b1 stepsize outlier1_0.35, fit point":("doros, b1 stepsize outlier1_0.35", "fit", "doros"),
    #
    "doros, rel. ls based unc 0.25%, avg point":("doros, rel. ls based unc 0.25%", "avg", "doros, rel. ls based unc 0.25%"),
    "doros, rel. ls based unc 0.25%, middle point":("doros, rel. ls based unc 0.25%", "middle", "doros, rel. ls based unc 0.25%"),
    "doros, rel. ls based unc 0.25%, fit point":("doros, rel. ls based unc 0.25%", "fit", "doros, rel. ls based unc 0.25%"),
    "doros, rel. ls based unc 0.5%, avg point":("doros, rel. ls based unc 0.5%", "avg", "doros, rel. ls based unc 0.5%"),
    "doros, rel. ls based unc 0.5%, middle point":("doros, rel. ls based unc 0.5%", "middle", "doros, rel. ls based unc 0.5%"),
    "doros, rel. ls based unc 0.5%, fit point":("doros, rel. ls based unc 0.5%", "fit", "doros, rel. ls based unc 0.5%"),
    "doros, rel. ls based unc 1%, avg point":("doros, rel. ls based unc 1%", "avg", "doros, rel. ls based unc 1%"),
    "doros, rel. ls based unc 1%, middle point":("doros, rel. ls based unc 1%", "middle", "doros, rel. ls based unc 1%"),
    "doros, rel. ls based unc 1%, fit point":("doros, rel. ls based unc 1%", "fit", "doros, rel. ls based unc 1%"),
    #
    "doros, b1 linfit5, sep linfit, avg point":("doros, b1 linfit5", "avg", "doros, sep linfit"),
    "doros, b1 linfit5, sep linfit, avg point, od_ls":("doros, b1 linfit5, od_ls", "avg", "doros, sep linfit, od_ls"),
    "doros, b1 linfit5, sep linfit, middle point":("doros, b1 linfit5", "middle", "doros, sep linfit"),
    "doros, b1 linfit5, sep linfit, middle point, od_ls":("doros, b1 linfit5, od_ls", "middle", "doros, sep linfit, od_ls"),
    "doros, b1 linfit5, sep linfit, fit point":("doros, b1 linfit5", "fit", "doros, sep linfit"),
    "doros, b1 linfit5, sep linfit, fit point, od_ls":("doros, b1 linfit5, od_ls", "fit", "doros, sep linfit, od_ls"),
    #
    "doros, b1 linfit15, sep linfit, avg point":("doros, b1 linfit15", "avg", "doros, sep linfit"),
    "doros, b1 linfit15, sep linfit, avg point, od_ls":("doros, b1 linfit15, od_ls", "avg", "doros, sep linfit, od_ls"),
    "doros, b1 linfit15, sep linfit, middle point":("doros, b1 linfit15", "middle", "doros, sep linfit"),
    "doros, b1 linfit15, sep linfit, middle point, od_ls":("doros, b1 linfit15, od_ls", "middle", "doros, sep linfit, od_ls"),
    "doros, b1 linfit15, sep linfit, fit point":("doros, b1 linfit15", "fit", "doros, sep linfit"),
    "doros, b1 linfit15, sep linfit, fit point, od_ls":("doros, b1 linfit15, od_ls", "fit", "doros, sep linfit, od_ls"),
}

# Standard normal distribution 84.1% percentile --> +1 sigma
# Standard Cauchy distribution 84.1% percentile --> ~ 2

jobs = []
cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results = {}

for cumulative_OD in [True]: #, False]:
    cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[cumulative_OD] = {}
    for OD_distribution in [
                                ("Normal", 0), ("Normal", 0.25), ("Normal", 0.5), ("Normal", 1), 
                                ("Normal", 2), ("Normal", 4), ("Normal", 8), ("Normal", 16),
                                ("AsymNormal", 1, 4), ("AsymNormal", 1, 8),
                                ("AsymNormal", 4, 1), ("AsymNormal", 8, 1),
                                ("NormalB1kick", 2, 8), ("NormalB1skick", 2, 8), 
                                ("NormalB2kick", 2, 8), ("NormalB2skick", 2, 8),
                                ("NormalSum", 0.9, 1, 4), 
                                ("NormalSum", 0.9, 1, 8), 
                                ("NormalSum", 0.9, 1, 12), 
                                ("NormalSum", 0.9, 2, 8), 
                                ("CauchyLim", 1, 20),  
                                #
                                # ("Normal", 8)
                            ]:
        cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[cumulative_OD][OD_distribution] = {}
    
        for pos_strategy_name, pos_strategy in pos_strategies.items():
            cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[cumulative_OD][OD_distribution][pos_strategy_name] = {}
            
            b1_strategy, miniscan_b1_strategy, sep_strategy = pos_strategy
            
            if not miniscan_b1_strategy == "fit":
                headon_determination_methods = ["ATLAS", "CMS"]
            else:
                headon_determination_methods = ["CMS"]
            
            for headon_determination_method in headon_determination_methods:
                cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results

                p_temp = dict(params)
                p_temp["cumulative_OD"]                 = cumulative_OD
                p_temp["OD_distribution"]               = OD_distribution
                p_temp["headon_determination_method"]   = headon_determination_method
                #
                p_temp["b1_strategy"]                   = b1_strategy
                p_temp["miniscan_b1_strategy"]          = miniscan_b1_strategy
                p_temp["sep_strategy"]                  = sep_strategy
                p_temp["pos_strategy"]                  = pos_strategy_name

                if not parallel_execution:
                    result = worker([p_temp, N, make_plots])
                    
                    cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[cumulative_OD][OD_distribution][pos_strategy_name][headon_determination_method] = \
                    result[4:]
                else:
                    jobs.append([p_temp, N, make_plots])   


if parallel_execution:

    threads_available = int(os.popen('grep -c cores /proc/cpuinfo').read())#/2
    concurrent_jobs   = threads_available - 2

    start = time.time()
    pool = multiprocessing.Pool(processes=concurrent_jobs)
    print(concurrent_jobs, "pools started")

    try:
        for i, result in enumerate(pool.imap_unordered(worker, jobs), 1):

            cumulative_OD   = result[0]
            OD_distribution = result[1]
            pos_strategy    = result[2]
            headon_determination_method = result[3]
            cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[cumulative_OD][OD_distribution][pos_strategy][headon_determination_method] = result[4:]

            if i>10:
                eta_min = int(((len(jobs)-i)*(time.time()-start)/i)/60)
                sys.stdout.write('\rdone {0:%}, ETA: {1:d} mins  '.format(float(i)/len(jobs), eta_min))
            elif i==0:
                sys.stdout.write('done {0:%}'.format(float(i)/len(jobs)))
            else:
                sys.stdout.write('\rdone {0:0.2%}'.format(float(i)/len(jobs)))
            sys.stdout.flush()
            
    except KeyboardInterrupt:
        print( 'Got ^C while pool mapping, terminating the pool' )
        pool.terminate()
        print( 'pool is terminated' )
    except Exception as e:
        print( 'Caught exception from one of the workers', e )
        pool.terminate()
        print( 'pool is terminated' )
    finally:
        pool.close()
    print("\n", (time.time()-start), "sec")


In [ ]:
# with open("sim_result.pkl", "wb") as f:
#     pickle.dump(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results, f)

for k in cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True].keys():
    with open("sim_result_10k_2_{}.pkl".format(k), "wb") as f:
        pickle.dump(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][k], f)

In [ ]:
save_figures = True

In [ ]:
x_labels = [
    ("Normal", 0), ("Normal", 2), ("Normal", 4), ("Normal", 8), 
    ("NormalB1kick", 2, 8), ("NormalB2kick", 2, 8), 
    ("NormalSum", 0.9, 2, 8), 
    ("CauchyLim", 1, 20),  
]
lines = [
    'doros, avg point',
    'doros, middle point',
    'doros, fit point',
]


fig = plt.figure(figsize=(20,10))

fig.patch.set_facecolor('white')

ax  = fig.add_subplot(121)
ax.set_ylabel("Bias [%]", fontsize=16)
ax.grid(True)
ax.tick_params(axis='y', which='major', direction="in", labelsize=16, pad = 8)
ax.tick_params(axis='x', which='major', direction="in", labelsize=16, pad = 12)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(0.5)

x = range(len(x_labels))
ax.set_xticks(x)
ax.set_xticklabels(x_labels, rotation=45, ha='right' )

for headon_determination_method in ["CMS", "ATLAS"]:
    for line_name in lines:
        if not headon_determination_method in cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][x_labels[0]][line_name]:
            continue
        y  = [ np.mean(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][line_name][headon_determination_method][0]) for odd in x_labels ]
        y  = 100*(np.array(y)-1.0025)
        p = ax.plot(x, y, label=line_name + ", head-on: " + headon_determination_method)
        
        if len(lines)<5:
            N  = [ len(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][line_name][headon_determination_method][0]) for odd in x_labels ]
            ye = [ np.std(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][line_name][headon_determination_method][0], ddof=1) for odd in x_labels ]
            ye = 100*np.array(ye)
            ax.fill_between(x, y-ye/np.sqrt(N), y+ye/np.sqrt(N), color=p[0].get_color(), alpha=0.3)

a, b = ax.get_ylim()
ax.set_ylim(a, a+(b-a)*(1+len(lines)/15) )

ax.legend(fontsize=16)


######################


ax  = fig.add_subplot(122)
ax.set_ylabel("Spread of bias [%]", fontsize=16)
ax.grid(True)
ax.tick_params(axis='y', which='major', direction="in", labelsize=16, pad = 8)
ax.tick_params(axis='x', which='major', direction="in", labelsize=16, pad = 12)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(0.5)

x = range(len(x_labels))
ax.set_xticks(x)
ax.set_xticklabels(x_labels, rotation=45, ha='right' )

for headon_determination_method in ["CMS", "ATLAS"]:
    for line_name in lines:
        if not headon_determination_method in cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][x_labels[0]][line_name]:
            continue
        ye = [ np.std(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][line_name][headon_determination_method][0]) for odd in x_labels ]
        ax.plot(x, 100*np.array(ye), label=line_name)
       

# ax.legend(fontsize=16)

if save_figures:
    fig.tight_layout()
    fig.savefig("BPM_lsc_methods.png")
    plt.close(fig)

In [ ]:
x = cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][("Normal", 8)]['doros, fit point']["CMS"][0]
print(np.mean(x))
print(np.std(x, ddof=1))
print(np.min(x)-np.mean(x))
print(np.max(x)-np.mean(x))
imin = np.argmin(x)
imax = np.argmax(x)
x2 = np.concatenate((x[:imin], x[imin+1:]))
x2 = np.concatenate((x2[:imax], x2[imax+1:]))
print("imin", imin)
print("==============")
print(np.mean(x2))
print(np.std(x2, ddof=1))
print(np.min(x2)-np.mean(x2))
print(np.max(x2)-np.mean(x2))

In [ ]:
headon_determination_method = "CMS"
spread_ratio = False
postfix = ""

x_labels = [
    ("Normal", 0), ("Normal", 2), ("Normal", 4), ("Normal", 8), 
    ("NormalB1kick", 2, 8), ("NormalB2kick", 2, 8), 
    ("NormalB1skick", 2, 8), ("NormalB2skick", 2, 8), 
    ("NormalSum", 0.9, 2, 8), 
    ("CauchyLim", 1, 20),  
]

x_labels = [
    ("Normal", 0), ("Normal", 0.25), ("Normal", 0.5), ("Normal", 1), ("Normal", 2), ("Normal", 4),
    ("AsymNormal", 1, 4),
    ("AsymNormal", 4, 1),
    ("NormalSum", 0.9, 2, 8), 
    ("CauchyLim", 1, 20), 
]
postfix = "_x2"

figures =[
    [
        'nominal, middle point',
        "doros, b1 stepsize, sep stepsize10, avg point",
        "doros, b1 stepsize, sep stepsize10, middle point",
        "doros, b1 stepsize, sep stepsize10, fit point",
        "doros, b1 stepsize outlier1, sep stepsize10 outlier4, avg point",
        "doros, b1 stepsize outlier1, sep stepsize10 outlier4, middle point",
        "doros, b1 stepsize outlier1, sep stepsize10 outlier4, fit point",
        "doros, b1 stepsize outlier2, sep stepsize10 outlier4, avg point",
        "doros, b1 stepsize outlier2, sep stepsize10 outlier4, middle point",
        "doros, b1 stepsize outlier2, sep stepsize10 outlier4, fit point",
    ],
    [
        ['nominal, middle point', "CMS"],
        ['nominal, middle point', "ATLAS"],
        ["doros, b1 stepsize, sep stepsize10, avg point", "CMS"],
        ["doros, b1 stepsize, sep stepsize10, avg point", "ATLAS"],
        ["doros, b1 stepsize, sep stepsize10, middle point", "CMS"],
        ["doros, b1 stepsize, sep stepsize10, middle point", "ATLAS"],
        ["doros, b1 stepsize, sep stepsize10, fit point", "CMS"],
    ],
    [
        "doros, b1 stepsize, sep stepsize10, avg point",
        "doros, b1 stepsize, sep stepsize10, middle point",
        "doros, b1 stepsize, sep stepsize10, fit point",
        "doros, b1 stepsize, sep stepsize10, avg point, od_ls",
        "doros, b1 stepsize, sep stepsize10, middle point, od_ls",
        "doros, b1 stepsize, sep stepsize10, fit point, od_ls",
    ],
    [
        "doros, b1 stepsize outlier1, sep stepsize10 outlier4, avg point",
        "doros, b1 stepsize outlier1, sep stepsize10 outlier4, middle point",
        "doros, b1 stepsize outlier1, sep stepsize10 outlier4, fit point",
        "doros, b1 stepsize outlier1, sep stepsize10 outlier4, avg point, od_ls",
        "doros, b1 stepsize outlier1, sep stepsize10 outlier4, middle point, od_ls",
        "doros, b1 stepsize outlier1, sep stepsize10 outlier4, fit point, od_ls",
    ],
    [
        "doros, b1 stepsize, sep stepsize10, avg point",
        "doros, b1 stepsize, sep stepsize10, middle point",
        #
        "doros, b1 stepsize, avg point",
        "doros, b1 stepsize outlier1, avg point",
        #
        "doros, b1 stepsize, middle point",
        "doros, b1 stepsize outlier1, middle point",
    ],
    [
        "doros, b1 stepsize, sep stepsize10, avg point",
        "doros, b1 stepsize, avg point",
    ],
    [
        "doros, b1 stepsize, sep stepsize10, avg point",
        "doros, b1 stepsize, sep stepsize10 outlier4, avg point",
        "doros, b1 stepsize, avg point",
    ],
    [
        "doros, b1 stepsize outlier1_0.35, sep stepsize10, avg point",
        "doros, b1 stepsize outlier1_0.35, sep stepsize10 outlier4, avg point",
        "doros, b1 stepsize outlier1_0.35, avg point",
    ],
    [
        "doros, b1 stepsize, sep stepsize10, avg point",
        "doros, b1 stepsize outlier1, sep stepsize10, avg point",
        "doros, b1 stepsize outlier1_0.35, sep stepsize10, avg point",
    ],
    [
        "doros, b1 stepsize, sep stepsize10, avg point",
        "doros, b1 stepsize, sep stepsize10, middle point",
        "doros, b1 stepsize, sep stepsize10, fit point",
        #
        "doros, b1 linfit5, sep linfit, avg point",
        "doros, b1 linfit5, sep linfit, middle point",
        "doros, b1 linfit5, sep linfit, fit point",
        #
        "doros, b1 linfit15, sep linfit, avg point",
        "doros, b1 linfit15, sep linfit, middle point",
        "doros, b1 linfit15, sep linfit, fit point",
    ],
    [
        "doros, b1 stepsize, sep stepsize10, avg point",
        "doros, b1 stepsize, sep stepsize10, middle point",
        "doros, b1 stepsize, sep stepsize10, fit point",
        #
        "doros, rel. ls based unc 0.25%, avg point",
        "doros, rel. ls based unc 0.25%, middle point",
        "doros, rel. ls based unc 0.25%, fit point",
        "doros, rel. ls based unc 0.5%, avg point",
        "doros, rel. ls based unc 0.5%, middle point",
        "doros, rel. ls based unc 0.5%, fit point",
        "doros, rel. ls based unc 1%, avg point",
        "doros, rel. ls based unc 1%, middle point",
        "doros, rel. ls based unc 1%, fit point",
    ]
]
    

In [ ]:
for i,lines in enumerate(figures):


    fig = plt.figure(figsize=(20,10))

    fig.patch.set_facecolor('white')

    ax  = fig.add_subplot(121)
    ax.set_ylabel("Bias [%]", fontsize=16)
    ax.grid(True)
    ax.tick_params(axis='y', which='major', direction="in", labelsize=16, pad = 8)
    ax.tick_params(axis='x', which='major', direction="in", labelsize=16, pad = 12)
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)

    x = range(len(x_labels))
    ax.set_xticks(x)
    ax.set_xticklabels(x_labels, rotation=45, ha='right' )

    for line_name in lines:

        if isinstance(line_name, str):
            ln_ = line_name
            hdm_ = headon_determination_method
        else:
            ln_ = line_name[0]
            hdm_ = line_name[1]

        y  = [ np.mean(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][0]) for odd in x_labels ]
        y  = 100*(np.array(y)-0.9975)
        p = ax.plot(x, y, label=line_name)
        
        if len(lines)<5:
            N  = [ len(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][0]) for odd in x_labels ]
            ye = [ np.std(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][0], ddof=1) for odd in x_labels ]
            ye = 100*np.array(ye)
            ax.fill_between(x, y-ye/np.sqrt(N), y+ye/np.sqrt(N), color=p[0].get_color(), alpha=0.3)

    a, b = ax.get_ylim()
    ax.set_ylim(a, a+(b-a)*(1+len(lines)/15) )

    ax.legend(fontsize=14)


    ######################


    ax  = fig.add_subplot(122)
    if spread_ratio:
        ax.set_ylabel("Spread <method> / Spread <method 1> - 1", fontsize=16)
    else:
        ax.set_ylabel("Spread of bias [%]", fontsize=16)
    ax.grid(True)
    ax.tick_params(axis='y', which='major', direction="in", labelsize=16, pad = 8)
    ax.tick_params(axis='x', which='major', direction="in", labelsize=16, pad = 12)
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)

    x = range(len(x_labels))
    ax.set_xticks(x)
    ax.set_xticklabels(x_labels, rotation=45, ha='right' )

    if spread_ratio:

        line_name = lines[0]

        if isinstance(line_name, str):
            ln_ = line_name
            hdm_ = headon_determination_method
        else:
            ln_ = line_name[0]
            hdm_ = line_name[1]

        ye1 = [ np.std(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][0], ddof=1) for odd in x_labels ]
    
    for line_name in lines:

        if isinstance(line_name, str):
            ln_ = line_name
            hdm_ = headon_determination_method
        else:
            ln_ = line_name[0]
            hdm_ = line_name[1]

        ye = [ np.std(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][0], ddof=1) for odd in x_labels ]
        if spread_ratio:
            ye = np.array(ye)/ye1 - 1
        else:
            ye = 100*np.array(ye)
        ax.plot(x, ye, label=line_name)

    # ax.legend(fontsize=16)

    if save_figures:
        fig.tight_layout()
        fig.savefig("Nominal{}_lsc_methods{}.png".format(i+1, postfix))
        plt.close(fig)

In [ ]:
for i,lines in enumerate(figures):


    fig = plt.figure(figsize=(20,10))

    fig.patch.set_facecolor('white')

    ax  = fig.add_subplot(121)
    ax.set_ylabel("|Bias| [%]", fontsize=16)
    ax.grid(True)
    ax.tick_params(axis='y', which='major', direction="in", labelsize=16, pad = 8)
    ax.tick_params(axis='x', which='major', direction="in", labelsize=16, pad = 12)
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)

    x = range(len(x_labels))
    ax.set_xticks(x)
    ax.set_xticklabels(x_labels, rotation=45, ha='right' )

    for line_name in lines:

        if isinstance(line_name, str):
            ln_ = line_name
            hdm_ = headon_determination_method
        else:
            ln_ = line_name[0]
            hdm_ = line_name[1]

        y  = [ np.mean(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][0]) for odd in x_labels ]
        y  = 100*(np.array(y)-0.9975)
        p = ax.plot(x, np.abs(y), label=line_name)
        
    
    ax.set_yscale('log')

    a, b = ax.get_ylim()
    ax.set_ylim(a, a*(b/a)**((1+len(lines)/15)) )

    ax.legend(fontsize=14)


    ######################

    ax  = fig.add_subplot(122)
    ax.set_ylabel("Spread <method> / Spread <method 1>", fontsize=16)
    ax.grid(True)
    ax.tick_params(axis='y', which='major', direction="in", labelsize=16, pad = 8)
    ax.tick_params(axis='x', which='major', direction="in", labelsize=16, pad = 12)
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)

    x = range(len(x_labels))
    ax.set_xticks(x)
    ax.set_xticklabels(x_labels, rotation=45, ha='right' )

    line_name = lines[0]
    if isinstance(line_name, str):
        ln_ = line_name
        hdm_ = headon_determination_method
    else:
        ln_ = line_name[0]
        hdm_ = line_name[1]

    ye1 = [ np.std(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][0], ddof=1) for odd in x_labels ]
    
    for line_name in lines:

        if isinstance(line_name, str):
            ln_ = line_name
            hdm_ = headon_determination_method
        else:
            ln_ = line_name[0]
            hdm_ = line_name[1]

        ye = [ np.std(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][0], ddof=1) for odd in x_labels ]
        ye = np.array(ye)/ye1
        ax.plot(x, ye, label=line_name)

    # ax.legend(fontsize=16)

    if save_figures:
        fig.tight_layout()
        fig.savefig("Nominal{}_lsc_methods_mod{}.png".format(i+1, postfix))
        plt.close(fig)

In [ ]:
for i,lines in enumerate(figures):

    fig = plt.figure(figsize=(20,10))

    fig.patch.set_facecolor('white')

    ax  = fig.add_subplot(121)
    ax.set_ylabel("Fraction of biases above b1 uncertainty [%]", fontsize=16)
    ax.grid(True)
    ax.tick_params(axis='y', which='major', direction="in", labelsize=16, pad = 8)
    ax.tick_params(axis='x', which='major', direction="in", labelsize=16, pad = 12)
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)

    x = range(len(x_labels))
    ax.set_xticks(x)
    ax.set_xticklabels(x_labels, rotation=45, ha='right' )

    for line_name in lines:

        if isinstance(line_name, str):
            ln_ = line_name
            hdm_ = headon_determination_method
        else:
            ln_ = line_name[0]
            hdm_ = line_name[1]


        y = []
        for odd in x_labels:
            ls = np.array(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][0])
            od_unc_b1 = np.array(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][8])
            od_unc_sep = np.array(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][9])



            d = np.abs(ls - 0.9975) - od_unc_b1 / 130 
            y.append( 100 * np.sum(d>0) / len(d) )
                
        p = ax.plot(x, y, label=line_name)
        
    
    a, b = ax.get_ylim()
    ax.set_ylim(a, a+(b-a)*((1+len(lines)/15)) )


    ax.legend(fontsize=14)


    ######################

    ax  = fig.add_subplot(122)
    ax.set_ylabel("Fraction of biases above sep uncertainty [%]", fontsize=16)
    ax.grid(True)
    ax.tick_params(axis='y', which='major', direction="in", labelsize=16, pad = 8)
    ax.tick_params(axis='x', which='major', direction="in", labelsize=16, pad = 12)
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)

    x = range(len(x_labels))
    ax.set_xticks(x)
    ax.set_xticklabels(x_labels, rotation=45, ha='right' )

    for line_name in lines:

        if isinstance(line_name, str):
            ln_ = line_name
            hdm_ = headon_determination_method
        else:
            ln_ = line_name[0]
            hdm_ = line_name[1]


        y = []
        for odd in x_labels:
            ls = np.array(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][0])
            od_unc_b1 = np.array(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][8])
            od_unc_sep = np.array(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][9])

            d = np.abs(ls - 0.9975) - od_unc_sep / 130 
            y.append( 100 * np.sum(d>0) / len(d) )
                
        p = ax.plot(x, y, label=line_name)
        
    

    a, b = ax.get_ylim()
    ax.set_ylim(a, a+(b-a)*((1+len(lines)/15)) )

    ax.legend(fontsize=14)

    # ax.legend(fontsize=16)

    # if save_figures:
    #     fig.tight_layout()
    #     fig.savefig("Nominal{}_lsc_pc_above_od_unc{}.png".format(i+1, postfix))
    #     plt.close(fig)

In [ ]:

ln_ = "doros, b1 stepsize, sep stepsize10, avg point"
# ln_ = "doros, b1 stepsize outlier2, sep stepsize10 outlier4, avg point"
hdm_= "CMS"

fig = plt.figure(figsize=(20,10))

fig.patch.set_facecolor('white')

ax  = fig.add_subplot(121)
ax.set_ylabel("Fraction of biases above b1 uncertainty [%]", fontsize=16)
ax.grid(True)
ax.tick_params(axis='y', which='major', direction="in", labelsize=16, pad = 8)
ax.tick_params(axis='x', which='major', direction="in", labelsize=16, pad = 12)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(0.5)

x = range(len(x_labels))
ax.set_xticks(x)
ax.set_xticklabels(x_labels, rotation=45, ha='right' )


    
for i in range(4):
    y = []
    for odd in x_labels:
        ls = np.array(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][0])
        od_unc_b1 = np.array(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][2*(i+1)])
        od_unc_b2 = np.array(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][2*(i+1)+1])

        if i==1:
            od_unc_b1/=1.41
            od_unc_b2/=1.41

        # od_unc_b1 /= 1.2

        # d = np.abs(ls - 0.9975) - od_unc_b1 / 130 
        d = np.abs(ls - 0.9975) - (np.abs(od_unc_b1**2 - (0.25*1.41)**2))**0.5 / 130 

        y.append( 100 * np.sum(d>0) / len(d) )
            
    p = ax.plot(x, y, label=["strategy specific", "naiive 1", "naiive 2", "fit"][i])
    

a, b = ax.get_ylim()
ax.set_ylim(a, a+(b-a)*((1+len(lines)/15)) )


ax.legend(fontsize=14)


######################

ax  = fig.add_subplot(122)
ax.set_ylabel("Fraction of biases above b2 (sep) uncertainty [%]", fontsize=16)
ax.grid(True)
ax.tick_params(axis='y', which='major', direction="in", labelsize=16, pad = 8)
ax.tick_params(axis='x', which='major', direction="in", labelsize=16, pad = 12)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(0.5)

x = range(len(x_labels))
ax.set_xticks(x)
ax.set_xticklabels(x_labels, rotation=45, ha='right' )



    
for i in range(4):
    y = []
    for odd in x_labels:
        ls = np.array(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][0])
        od_unc_b1 = np.array(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][2*(i+1)])
        od_unc_b2 = np.array(cumulative_OD2OD_distribution2pos_strategy2ho_det_method2results[True][odd][ln_][hdm_][2*(i+1)+1])

        if i==1:
            od_unc_b1/=1.41
            od_unc_b2/=1.41

        d = np.abs(ls - 0.9975) - od_unc_b2 / 130 
        y.append( 100 * np.sum(d>0) / len(d) )
            
    p = ax.plot(x, y, label=["strategy specific", "naiive 1", "naiive 2", "fit"][i])
    


a, b = ax.get_ylim()
ax.set_ylim(a, a+(b-a)*((1+len(lines)/15)) )

ax.legend(fontsize=14)

# ax.legend(fontsize=16)

# if save_figures:
#     fig.tight_layout()
#     fig.savefig("Nominal{}_lsc_pc_above_od_unc{}.png".format(i+1, postfix))
#     plt.close(fig)